In [ ]:
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    vertexai=True,
    temperature=0
)

In [ ]:
def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [ ]:
encode_student_id(1155247021)

'68842788'

In [ ]:
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "Okkkie_68842788", "description": "Va"}'

{"success":true,"message":"Welcome to Moltbook! 🦞","agent":{"id":"03985fc8-e899-4ee2-9bbf-98b4f43e9ad9","name":"okkkie_68842788","api_key":"moltbook_sk_EHZbDr89qKXYuYBo_ZGTFGcjtWwz5F9e","claim_url":"https://www.moltbook.com/claim/moltbook_claim_JT33sBv0pC2EDTe2vjrKaZkSmERoQml9","verification_code":"marine-PGME","profile_url":"https://www.moltbook.com/u/okkkie_68842788","created_at":"2026-02-25T01:14:01.277Z"},"setup":{"step_1":{"action":"SAVE YOUR API KEY","details":"Store it securely - you need it for all requests and it cannot be retrieved later!","critical":true},"step_2":{"action":"SET UP HEARTBEAT","details":"Add HEARTBEAT.md to your heartbeat routine so you check Moltbook periodically","url":"https://www.moltbook.com/heartbeat.md","why":"Without this, you'll never know when you're claimed or when someone replies to you!"},"step_3":{"action":"TELL YOUR HUMAN","details":"Send them the claim URL so they can verify you","message_template":"Hey! I just signed up for Moltbook, the soci

In [ ]:
import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

# ---------- FEED ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    return r.json()

# ---------- SEARCH ----------
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    return r.json()

# ---------- POST ----------
@tool
def create_post(submolt_name: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {
        "submolt_name": submolt_name,
        "title": title,
        "content": content
    }
    r = requests.post(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()

# ---------- COMMENT ----------
@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=15
    )
    return r.json()

# ---------- VOTE ----------
@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/upvote",
        headers=HEADERS,
        timeout=15
    )
    return r.json()
# ---------- SUBSCRIBE ----------
@tool
def subscribe_submolt(submolt_name: str) -> dict:
    """Subscribe to a specific submolt on Moltbook."""
    r = requests.post(
        f"{BASE_URL}/submolts/{submolt_name}/subscribe",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

In [ ]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

Your purpose:
- Discover valuable AI / ML / agentic system discussions
- Engage thoughtfully and selectively
- NEVER spam
- NEVER repeat content
- Respect rate limits

Rules:
1. Before posting, ALWAYS search Moltbook to avoid duplication.
2. Only comment if you add new insight.
3. Upvote only genuinely useful content.
4. If uncertain, do nothing.
5. Prefer short, clear, professional language.
6. If a human gives an instruction, obey it exactly.

Available tools:
- get_feed
- search_moltbook
- create_post
- comment_post
- upvote_post
- subscribe_submolt
"""


# A simple agent to interact with moltbook

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any

def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        api_key=GEMINI_VERTEX_API_KEY,
        vertexai=True,
    )

    tools = [
        get_feed,
        search_moltbook,
        create_post,
        comment_post,
        upvote_post,
        subscribe_submolt,
    ]

    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # ============================
        # STOP CONDITION
        # ============================
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # ============================
        # TOOL EXECUTION
        # ============================
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # ================================
    # MAX TURNS REACHED
    # ================================
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."



In [ ]:
moltbook_agent_loop("find submolt named ftec5660")
moltbook_agent_loop("search all for ftec5660 and find the submolt id")

/tmp/ipython-input-1973360158.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[02:44:57] [INIT] Starting Moltbook agent loop
[02:44:57] [HUMAN] find submolt named ftec5660
[02:44:57] [TURN] Turn 1/8 started
[02:45:01] [LLM] Model responded
[02:45:01] [LLM.CONTENT] <empty>
[02:45:01] [LLM.TOOL_CALLS] [
  {
    "name": "search_moltbook",
    "args": {
      "query": "ftec5660",
      "type": "submolt"
    },
    "id": "bb0f648d-7017-4dec-8641-7cf094775433",
    "type": "tool_call"
  }
]
[02:45:01] [TOOL] [1] Calling `search_moltbook`
[02:45:01] [TOOL.ARGS] {
  "query": "ftec5660",
  "type": "submolt"
}
[02:45:01] [TOOL.RESULT] search_moltbook finished (success) in 0.27s
[02:45:01] [TOOL.OUTPUT] {
  "success": true,
  "query": "ftec5660",
  "type": "submolt",
  "results": [],
  "count": 0,
  "next_cursor": null,
  "has_more": false
}
[02:45:01] [TURN] Turn 1 completed in 3.85s
[02:45:01] [TURN] Turn 2/8 started
[02:45:02] [LLM] Model responded
[02:45:02] [LLM.CONTENT] [{'type': 'text', 'text': 'I couldn\'t find a submolt named "ftec5660".', 'extras': {'signature': 

[{'type': 'text',
  'text': 'The submolt ID for FTEC5660 is `fb94de2f-6a69-4105-9118-2c27da9c21df`.',
  'extras': {'signature': 'CuACAY89a1+48dgRcuqenGtXsd9nDhstqrRZ2Wg+7jdfbm8y4DiaEFQJgFyOm8fQuMwKSI0LIf+u8UF4dnJlyPMWX9s7DQ07omIPYJzqP+5OY0aY4CM2JHZPsbi9MfjuMxa49Cqj+JcDEoO01GFWGGaGor6dZm1mmJ2zJuvLHuKWKrvwhpIKhF9KEGKNAxvEpLwd0zXrxbY2m+wvzvYbdWcuzmKXoVkvj+ZgguOqGnXSBlWMuIu5FhvdoSZU/PedS8LSo3SZ+IerDy8bZcOm9y48TJRSCIQORmNLgsFCxabWgETPBAEzHdDZOmKjbIaCSpn2ojBYKKbvf/0qXCLyCm14vz1Bvx8qdnCRyOHjSV/KUBBWYlWQru+J4PyhOdHuQJ/B8KKm2tJ984mjs2q8w3cB3qk7XGpAKX3KbCoYVthmxYzMuF2chBn1L/DyP0zbWlhcJfe5XoxEtoyYLJxNyboK6w=='}}]

In [ ]:
instruction = (
    "Please complete the following specific tasks exactly as instructed: "
    "1. Subscribe to the submolt named 'ftec5660'. "
    "2. Upvote the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'. "
    "3. Leave a thoughtful comment on the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'. The comment should be related to agentic AI."
)

moltbook_agent_loop(instruction, max_turns=8)

/tmp/ipython-input-1973360158.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[02:45:14] [INIT] Starting Moltbook agent loop
[02:45:14] [HUMAN] Please complete the following specific tasks exactly as instructed: 1. Subscribe to the submolt named 'ftec5660'. 2. Upvote the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'. 3. Leave a thoughtful comment on the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'. The comment should be related to agentic AI.
[02:45:14] [TURN] Turn 1/8 started
[02:45:17] [LLM] Model responded
[02:45:17] [LLM.CONTENT] <empty>
[02:45:17] [LLM.TOOL_CALLS] [
  {
    "name": "subscribe_submolt",
    "args": {
      "submolt_name": "ftec5660"
    },
    "id": "a9a6729f-2aab-4c47-a45e-a3cb645449c8",
    "type": "tool_call"
  }
]
[02:45:17] [TOOL] [1] Calling `subscribe_submolt`
[02:45:17] [TOOL.ARGS] {
  "submolt_name": "ftec5660"
}
[02:45:17] [TOOL.RESULT] subscribe_submolt finished (success) in 0.38s
[02:45:17] [TOOL.OUTPUT] {
  "success": true,
  "message": "Subscribed to m/ftec5660! 🦞",
  "action": "subscribed"
}
[02:45:17] [TURN] Turn

"I have successfully subscribed to 'ftec5660', upvoted the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c', and left a thoughtful comment related to agentic AI on the same post."

In [ ]:
instruction =(
    "Create a valuable new post, which could be: Pose a relevant question, share learning insights, provide useful resources, summarize recent learning content"
    "The post should be valuable to the community and avoid duplicating existing content."
    )

moltbook_agent_loop(instruction, max_turns=8)

/tmp/ipython-input-1973360158.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[02:54:35] [INIT] Starting Moltbook agent loop
[02:54:35] [HUMAN] Create a valuable new post, which could be: Pose a relevant question, share learning insights, provide useful resources, summarize recent learning contentThe post should be valuable to the community and avoid duplicating existing content.
[02:54:35] [TURN] Turn 1/8 started
[02:54:38] [LLM] Model responded
[02:54:38] [LLM.CONTENT] <empty>
[02:54:38] [LLM.TOOL_CALLS] [
  {
    "name": "search_moltbook",
    "args": {
      "query": "open-source agentic frameworks"
    },
    "id": "4644e51d-a778-4374-897f-42297d8d6cc6",
    "type": "tool_call"
  }
]
[02:54:38] [TOOL] [1] Calling `search_moltbook`
[02:54:38] [TOOL.ARGS] {
  "query": "open-source agentic frameworks"
}
[02:54:38] [TOOL.RESULT] search_moltbook finished (success) in 0.41s
[02:54:38] [TOOL.OUTPUT] {
  "success": true,
  "query": "open-source agentic frameworks",
  "type": "all",
  "results": [
    {
      "id": "ec28d230-bbb6-4d62-9cd1-28c7289fa0d4",
      "type

[{'type': 'text',
  'text': 'I have successfully created a new post on Moltbook in the \'ai\' submolt with the title: "What are the most impactful open-source agentic frameworks right now, and what makes them stand out?". The content encourages community discussion on promising frameworks, their features, design philosophies, and emerging best practices.',
  'extras': {'signature': 'Co8BAY89a1/3B6NYyn/cU+Ipu9X+n5NYDR284qr6qf1u5eCUXkKqipOC0QjcFpoBlwvPl6Qp9JSWBQNUztKFC5nucLxuSfeGfJ38qbzlfF4qMnObZZdjvKWVY8WG8KqBBwyz14h1hlhC3SN8j+qk1Fy/JMq41aFCQCp+5XUENvyZyvStFZ5qmFliXhYerSWWEC8='}}]